## User Instructions
This is a program which will create R-Maps. These are used to correlate a voxel to a continuous outcome measure.
As it stands, this software employs Pearson Correlation Coefficients, which imply it will be best to have a continuous outcome on a percent scale. If you would like to do otherwise, a Spearman Correlation is possible. 

The software will walk you through everything. 

Files are expected to follow a BIDS naming convention. 
Subject IDs are expected in the input CSV and are expected to share the same naming convention as the nifti files themselves.

In [ ]:
import pandas as pd
import numpy as np
from calvin_utils.file_utils.import_functions import paths_to_input_files
# What is the path to the folder/csv containing the nifti files/files paths for the neuroimaging files?
path_1 = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/corbetta/BIDS_Dataset'

# What is the path to the csv with your outcome measures? Please note:
## This software expects csv to contain an outcome in a column with subject names in another column.
path_2 = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/corbetta/Study_Metadata/Behavior_FactorAnalysis_week.csv'

# What is would you like to call this analysis? 
## This will generate a subfolder called analysis_name at the location of path_1 and save all outputs there.
analysis_name = 'corbetta_analysis'

#----------------------------------------------------------------User Input Above----------------------------------------------------------------
path_1, path_2, out_dir = paths_to_input_files(
    path_1 = path_1,
    path_2 = path_2,
    analysis_name = analysis_name
)

In [ ]:
import os
from calvin_utils.import_matrices import import_matrices_from_folder, csv_of_nifti_filepaths_to_dataframe

#Do you want to import from folder or csv?
import_choice = 'folder'

#----------------------------------------------------------------
if import_choice == 'folder':
    df_1 = import_matrices_from_folder(path_1, file_pattern='/*/*/*stat-t_conn.nii*')
elif import_choice == 'csv':
    df_1 = csv_of_nifti_filepaths_to_dataframe(path_1)
else:
    raise ValueError('Unexpected import choice: %s' % import_choice)
df_1 = df_1.transpose()

In [ ]:
path_list = df_1.index.to_list()
basename_path_list = [os.path.basename(path).split('_')[0] for path in path_list]
df_1['subject'] = basename_path_list
df_1

In [ ]:
# Import a CSV with the clinical data of interest
df_2 = pd.read_csv(path_2)
df_2

In [ ]:
import os
# What is the name of the column that contains your subject labels?
subject_column = 'study_id'
# What you should see below is a dataframe with subjects as rows, clinical outcome/data in the first columns, and then voxel values thereafter.
column_to_merge_on = 'AttentionVF'
#----------------------------------------------------------------Input Above----------------------------------------------------------------
path_list = df_2.loc[:, subject_column].to_list()
try:
    basename_path_list = [os.path.basename(path).split('_')[1] for path in path_list]
except:
    basename_path_list = [str(path) for path in path_list]
df_2['subject'] = basename_path_list
df_2['subject']

In [ ]:
# Let's take a look at some descriptive statistics
df_2.describe().transpose()

In [ ]:
# Now, join voxels to the clinical data

#----------------------------------------------------------------
prep_df = df_2.copy()
final_df = df_2.loc[:, [column_to_merge_on, 'subject']].merge(df_1, on='subject', right_index=False)
merged_df = final_df.set_index('subject')
display(merged_df)

In [ ]:
# Enter the names of any columns you would like to remove from your dataframe below.
# If you do not want to remove columns, leave as columns_to_remove = None
# If you want to remove columns from the dataframe, leave as columns_to_remove = ['colulmn_1_name', 'colulmn_2_name', etc.]

columns_to_remove = None

#----------------------------------------------------------------User Inputs----------------------------------------------------------------
final_df = merged_df.copy()

#Remove undesirable columns
try:
    final_df = final_df.dropna()
    final_df.pop(columns_to_remove)
except:
    print('Soft error raised: could not find column %s:' % columns_to_remove)
display(final_df)

In [ ]:
from calvin_utils.network_mapping_utilities import generate_r_map
r_df, p_df, r_squared_df = generate_r_map(final_df, mask_path=None)

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(r_df, (out_dir+f'/r_map/{column_to_merge_on}'))

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(p_df, (out_dir+f'/p_map/{column_to_merge_on}'))

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(r_squared_df, (out_dir+f'/r_squared_map/{column_to_merge_on}'))

## Perform Delta R-Map and Permute it for Significance

## Calculate the Observed Delta-R Map Between 2 Populations

In [ ]:
from calvin_utils.network_mapping_utilities import generate_delta_r_map
delta_matrix = merged_df.copy()
observed_delta_r_map = generate_delta_r_map(delta_matrix, threshold_of_interest=65, column_of_interest='Age at DOS')

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(observed_delta_r_map, (out_dir+'/over_vs_under_65_delta_r_map'))

## Calculate the Empiric Delta-R Map Distribution 
### Note, this permutes the label of the population without permuting the neuroimaging data.
### Therefore, we are testing if the separation of the r-maps is significantly due to the variable of interest. 

In [ ]:
from calvin_utils.network_mapping_utilities import permuted_patient_label_delta_r_map
from calvin_utils.print_suppression import HiddenPrints
n_permutations = 2
column_of_interest = 'Age at DOS'
threshold_of_interest = 65
with HiddenPrints():
    p_count_df = permuted_patient_label_delta_r_map(dataframe_to_permute=merged_df, 
                                                observed_delta_r_map=observed_delta_r_map, 
                                                column_of_interest=column_of_interest, 
                                                threshold_of_interest=threshold_of_interest, 
                                                n_permutations=n_permutations)

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(p_values_df, (out_dir+'/over_vs_under_65_delta_r_map_p_values_df'))

In [ ]:
merged_df.pop('Patient # CDR, ADAS')
# merged_df.to_csv(os.path.join(out_dir, 'csv_for_delta_r_permutation.csv'))

In [ ]:
merged_df.pop('Patient # CDR, ADAS')
merged_df.to_csv(os.path.join(out_dir, 'csv_for_permuted_delta_r_map.csv'), index=False)